In [1]:
import os
import shutil
import pandas as pd
from tabulate import tabulate
from docx import Document

from docx.shared import Pt
from docx.oxml.ns import qn   # ✅ 이거 꼭 필요함
import re

from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_ALIGN_VERTICAL
import re


In [2]:
# =========================
# Word 파일 복사 함수
# =========================
def copy_word_with_exam_number(df, row_index, word_template_path):
    """
    Word 템플릿 파일에서 'GS-X-XX-0XXX' 부분을
    df의 '시험번호' 값으로 치환하여 복사본 생성

    Args:
        df (pd.DataFrame): 시험번호가 들어있는 데이터프레임
        row_index (int): 몇 번째 행의 시험번호를 쓸지
        word_template_path (str): 원본 Word 파일 경로

    Returns:
        str: 새 Word 파일 경로
    """
    exam_number = df.loc[row_index, "시험번호"]

    # 경로 분리
    dir_name, file_name = os.path.split(word_template_path)

    # 파일명에서 "GS-X-XX-0XXX"만 교체 (샘플_영남은 그대로 유지됨)
    new_file_name = file_name.replace("GS-X-XX-0XXX", exam_number)
    new_word_path = os.path.join(dir_name, new_file_name)

    # 복사본 생성
    shutil.copy(word_template_path, new_word_path)

    print(f"📄 복사본 생성 완료: {new_word_path}")
    return new_word_path



def update_word_header(df, row_index, word_file_path):
    """
    Word 문서의 머리글에서 접수번호 / 성적서번호를
    DataFrame 값으로 치환하고 글꼴을 맑은 고딕, 크기를 9pt로 고정
    """
    # DataFrame 값 읽기
    receipt_number = str(df.loc[row_index, "접수번호"])       # 예: 24-01910
    exam_number = "T" + str(df.loc[row_index, "시험번호"])   # 예: TGS-C-24-0046

    # Word 문서 열기
    doc = Document(word_file_path)

    # 첫 번째 섹션의 헤더 접근
    header = doc.sections[0].header

    # 문단 치환 + 서식 적용
    for para in header.paragraphs:
        if "접수번호" in para.text:
            para.text = para.text.replace("24-05629", receipt_number)
        if "성적서 번호" in para.text or "성적서번호" in para.text:
            para.text = para.text.replace("TGS-A-24-0353", exam_number)

        for run in para.runs:
            run.font.name = "맑은 고딕"
            run._element.rPr.rFonts.set(qn('w:eastAsia'), "맑은 고딕")  # ✅ 한글 폰트 적용
            run.font.size = Pt(9)

    # 머리글 안의 표도 처리
    for tbl in header.tables:
        for row in tbl.rows:
            for cell in row.cells:
                for para in cell.paragraphs:
                    if "접수번호" in para.text:
                        para.text = para.text.replace("24-05629", receipt_number)
                    if "성적서 번호" in para.text or "성적서번호" in para.text:
                        para.text = para.text.replace("TGS-A-24-0353", exam_number)

                    for run in para.runs:
                        run.font.name = "맑은 고딕"
                        run._element.rPr.rFonts.set(qn('w:eastAsia'), "맑은 고딕")
                        run.font.size = Pt(9)

    # 저장
    doc.save(word_file_path)
    print(f"✏️ 머리글 수정 완료 (맑은 고딕, 9pt 적용): {word_file_path}")

# 표의 여러 항목을 df 값으로 치환 (굴림 10pt, 가로/세로 중앙정렬)
def update_table_receipt_date(df, row_index, word_file_path):
    """
    표의 여러 라벨 값을 df 값으로 치환 (매핑: {기존라벨: 새값})
    - 라벨|값 2열 구조: 라벨 셀을 찾아 오른쪽(또는 아래) 값 셀을 새값으로 교체
    - '라벨: 값' 한 셀 구조: 셀 전체를 '원본라벨: 새값'으로 교체 (원본 라벨 유지)
    - 스타일: 굴림 10pt, 가로/세로 중앙, 문단 간격 0
    """
    # ---- df 값 준비 ----
    v_writer   = str(df.loc[row_index, "작성자(PL)"])
    v_receipt  = str(df.loc[row_index, "접수번호"])
    v_org      = str(df.loc[row_index, "업체명.1"])
    v_sample   = str(df.loc[row_index, "시료명"])
    v_recvdate = str(df.loc[row_index, "접수일자"])  # 포맷은 이미 전처리됨

    # ---- 매핑: {기존라벨: 새값} ----
    mapping = {
        "작성자":    v_writer,    # "작성자 : 김책임" -> df['작성자(PL)']
        "기술책임자": "-",        # "기술책임자 : 박기책" -> "-"
        "접수번호":   v_receipt,   # "접수번호 : 24-05629" -> df['접수번호']
        "시험일자":   "-",        # "시험일자 : ... ~ ..." -> "-"
        "의뢰기관":   v_org,      # "의뢰기관 : ㈜우리소프트" -> df['업체명.1']
        "시료명":     v_sample,   # "시료명 : 우리소프트 V1.0." -> df['시료명']
        "접수일자":   v_recvdate, # 기존 접수일자 라벨도 처리
    }

    # '라벨: 값' 형태 매칭 정규식 (원본 라벨을 보존하려고 사용)
    label_value_pat = re.compile(r"^\s*([^\:：]+?)\s*[:：]\s*(.+?)\s*$")

    def canon_label(text: str) -> str:
        # 비교용 라벨 키: 공백/콜론 제거 후 축소
        return re.sub(r"[\s:：]", "", text).strip()

    def apply_style(cell):
        # 세로 중앙
        cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        for p in cell.paragraphs:
            # 가로 가운데
            p.alignment = WD_ALIGN_PARAGRAPH.CENTER
            # 문단 간격 제거
            if p.paragraph_format is not None:
                p.paragraph_format.space_before = Pt(0)
                p.paragraph_format.space_after = Pt(0)
            # 런 서식 적용
            for run in p.runs:
                run.font.name = "굴림"
                # 한글 폰트 적용
                run._element.rPr.rFonts.set(qn('w:eastAsia'), "굴림")
                run.font.size = Pt(10)

    doc = Document(word_file_path)

    for table in doc.tables:
        rows = table.rows
        for r, row in enumerate(rows):
            cells = row.cells
            for c, cell in enumerate(cells):
                raw = cell.text.replace("\r", "").replace("\n", "").strip()
                if not raw:
                    continue

                # 1) '라벨: 값' 한 셀 구조 처리 (원본 라벨 보존)
                m = label_value_pat.match(raw)
                if m:
                    orig_label = m.group(1).strip()
                    key = canon_label(orig_label)
                    if key in mapping:
                        new_value = mapping[key]
                        cell.text = f"{orig_label}: {new_value}"
                        apply_style(cell)
                    continue

                # 2) 라벨만 있는 셀(2열 구조) 처리
                key = canon_label(raw)
                if key in mapping:
                    # 타깃 셀: 오른쪽 우선, 없으면 아래
                    target_cell = None
                    if c + 1 < len(cells):
                        target_cell = cells[c + 1]
                    elif r + 1 < len(rows):
                        target_cell = table.cell(r + 1, c)

                    # 값 있는 셀이 있으면 새값으로 대체하고 스타일 적용
                    if target_cell is not None:
                        new_value = mapping[key]
                        target_cell.text = str(new_value)
                        apply_style(target_cell)

                    # 라벨 셀은 원래 텍스트 유지 (요청: 라벨명 변경 대신 값만 매핑)
                    # 만약 라벨명을 바꾸려면 여기서 cell.text = 변경값 으로 처리
                    continue

    doc.save(word_file_path)
    print(f"📝 표 항목 치환 완료(굴림 10pt, 중앙정렬): {word_file_path}")

    
# def update_product_info(df, row_index, word_file_path):
#     """
#     표/문단에서 아래 항목을 대체하고, 글꼴을 '굴림' 10pt로 적용
#       - 제품명 : -> df['시료명 1'] (없으면 df['시료명'])
#       - 버전   : -> df['버전']
#       - 'GS-... 점검표' 텍스트 -> '{시험번호} 점검표' (시험번호는 df['시험번호'])

#     사용:
#       update_product_info(df, i, new_path)
#     """
#     # --- 값 준비 (안정적으로 가져오기) ---
#     product = df.get("시료명 1", None)
#     if product is None:
#         product = df.get("시료명", "")
#     version = df.get("버전", "")
#     exam_no = df.get("시험번호", "")
#     # str 변환 (NaN 대비)
#     product = "" if pd.isna(product) else str(product)
#     version = "" if pd.isna(version) else str(version)
#     exam_no  = "" if pd.isna(exam_no) else str(exam_no)

#     # target strings
#     product_val = product
#     version_val = version
#     title_new = f"{exam_no} 점검표" if exam_no else " 점검표"

#     # 정규식들
#     label_value_pat = re.compile(r"^\s*([^\:：]+?)\s*[:：]\s*(.+?)\s*$")
#     gs_title_pat = re.compile(r"T?GS-[A-Z]-\d{2}-\d{4}\s*점검표", flags=re.IGNORECASE)  # 기존 패턴

#     def apply_gulim_10pt_to_cell(cell):
#         cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
#         for p in cell.paragraphs:
#             p.alignment = WD_ALIGN_PARAGRAPH.CENTER
#             if p.paragraph_format is not None:
#                 p.paragraph_format.space_before = Pt(0)
#                 p.paragraph_format.space_after = Pt(0)
#             for run in p.runs:
#                 run.font.name = "굴림"
#                 run._element.rPr.rFonts.set(qn('w:eastAsia'), "굴림")
#                 run.font.size = Pt(10)

#     def apply_gulim_10pt_to_run(run):
#         run.font.name = "굴림"
#         run._element.rPr.rFonts.set(qn('w:eastAsia'), "굴림")
#         run.font.size = Pt(10)

#     doc = Document(word_file_path)

#     # -------------------------
#     # 1) 테이블 내부 치환 (라벨|값 구조 또는 '라벨: 값' 단일셀)
#     # -------------------------
#     for table in doc.tables:
#         rows = table.rows
#         for r, row in enumerate(rows):
#             cells = row.cells
#             for c, cell in enumerate(cells):
#                 raw = cell.text.replace("\r", "").replace("\n", "").strip()
#                 if not raw:
#                     continue

#                 # 한 셀에 '라벨: 값' 구조인 경우 (원본 라벨 유지, 값 교체)
#                 m = label_value_pat.match(raw)
#                 if m:
#                     orig_label = m.group(1).strip()
#                     key = re.sub(r"[\s:：]", "", orig_label)
#                     # 제품명
#                     if key == "제품명":
#                         cell.text = f"{orig_label}: {product_val}"
#                         apply_gulim_10pt_to_cell(cell)
#                         continue
#                     # 버전
#                     if key == "버전":
#                         cell.text = f"{orig_label}: {version_val}"
#                         apply_gulim_10pt_to_cell(cell)
#                         continue
#                     # 'GS-... 점검표' 같은 경우도 셀 내 텍스트 치환
#                     # (라벨: 값 구조가 아니어도 아래서 처리)
                
#                 # 라벨만 있는 2열 구조: 오른쪽(또는 아래) 값 셀을 바꿈
#                 key_compact = re.sub(r"[\s:：]", "", raw)
#                 if key_compact == "제품명":
#                     # 값 셀 찾기
#                     target_cell = None
#                     if c + 1 < len(cells):
#                         target_cell = cells[c + 1]
#                     elif r + 1 < len(rows):
#                         target_cell = table.cell(r + 1, c)
#                     if target_cell is not None:
#                         target_cell.text = product_val
#                         apply_gulim_10pt_to_cell(target_cell)
#                     continue
#                 if key_compact == "버전":
#                     target_cell = None
#                     if c + 1 < len(cells):
#                         target_cell = cells[c + 1]
#                     elif r + 1 < len(rows):
#                         target_cell = table.cell(r + 1, c)
#                     if target_cell is not None:
#                         target_cell.text = version_val
#                         apply_gulim_10pt_to_cell(target_cell)
#                     continue
#                 # 'GS-... 점검표'가 라벨 셀에 존재하는 경우 (단독 텍스트)
#                 if gs_title_pat.search(raw):
#                     # 라벨 셀 자체를 바꿈(기존 라벨 보존 필요없으면 전체 교체)
#                     cell.text = gs_title_pat.sub(title_new, raw)
#                     apply_gulim_10pt_to_cell(cell)
#                     continue

#     # -------------------------
#     # 2) 문단(본문) 내 'GS-... 점검표' 텍스트 치환 (런 단위로 치환, 서식 유지)
#     # -------------------------
#     for para in doc.paragraphs:
#         for run in para.runs:
#             if gs_title_pat.search(run.text):
#                 run.text = gs_title_pat.sub(title_new, run.text)
#             # 제품명/버전이 문단 내에 '라벨: 값' 형식으로 존재하면 교체
#             if re.search(r"제품명\s*[:：]", run.text):
#                 run.text = re.sub(r"(제품명\s*[:：]\s*)(.+)", rf"\1{product_val}", run.text)
#             if re.search(r"버전\s*[:：]", run.text):
#                 run.text = re.sub(r"(버전\s*[:：]\s*)(.+)", rf"\1{version_val}", run.text)
#             # 서식 적용
#             apply_gulim_10pt_to_run(run)

#     # -------------------------
#     # 3) 테이블 내부의 문단(런)에도 추가로 서식 적용 (안정성)
#     # -------------------------
#     for table in doc.tables:
#         for row in table.rows:
#             for cell in row.cells:
#                 apply_gulim_10pt_to_cell(cell)

#     doc.save(word_file_path)
#     print(f"✅ 제품정보/버전/타이틀 치환 완료: {word_file_path}")

In [3]:
import re
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn

# --- 설정 및 상수 ---
FONT_NAME = "굴림"
FONT_SIZE = Pt(10)
REPLACEMENT_KEYS = ["제품명", "버전"] # 관리가 필요한 키 목록

def update_product_info_final(df, row_index, word_file_path):
    """
    표/문단에서 지정된 항목의 텍스트를 대체하고, 해당 텍스트의 글꼴만 '굴림' 10pt로 변경합니다.
    기존의 정렬, 들여쓰기 등 다른 서식은 모두 그대로 유지합니다.
      - 제품명, 버전 -> df 값으로 대체
      - 'GS-...' 점검표/결함리포트 -> '시험번호' 점검표/결함리포트로 대체
    """
    replacement_values = _get_data_from_df(df, row_index)
    
    # '점검표' 또는 '결함리포트'를 모두 찾는 정규식
    gs_report_pat = re.compile(r"(T?GS-[A-Z]-\d{2}-\d{4}\s*)(점검표|결함리포트)", re.IGNORECASE)

    doc = Document(word_file_path)
    _update_content_and_apply_style(doc, replacement_values, gs_report_pat)
    doc.save(word_file_path)
    print(f"✅ (최종 수정) 제품정보/버전/타이틀 치환 및 서식 적용 완료: {word_file_path}")


def _get_data_from_df(df, row_index):
    """DataFrame의 특정 행에서 데이터를 안전하게 추출하여 딕셔너리로 반환합니다."""
    row = df.iloc[row_index]
    product = row.get("시료명 1")
    if pd.isna(product):
        product = row.get("시료명", "")
    version = row.get("버전", "")
    exam_no = row.get("시험번호", "")
    return {
        "제품명": "" if pd.isna(product) else str(product),
        "버전": "" if pd.isna(version) else str(version),
        "exam_no": "" if pd.isna(exam_no) else str(exam_no),
    }

def _apply_font_style_only(run):
    """Run 객체에 글꼴과 크기만 적용합니다. (정렬 등은 변경하지 않음)"""
    run.font.name = FONT_NAME
    run._element.rPr.rFonts.set(qn('w:eastAsia'), FONT_NAME)
    run.font.size = FONT_SIZE

def _update_content_and_apply_style(doc, values, gs_report_pat):
    """문서 내 텍스트를 교체하고, 교체된 부분에만 즉시 서식을 적용합니다."""
    
    for para in _iter_paragraphs(doc):
        # ★★★ 변경점: Run이 아닌 Paragraph 전체 텍스트를 기준으로 검색 ★★★
        
        # --- 'GS-...' 점검표/결함리포트 치환 (문단 레벨) ---
        if gs_report_pat.search(para.text):
            # 기존 정렬 상태 저장
            original_alignment = para.alignment
            
            # 치환될 텍스트 생성 (re.sub는 모든 발생 패턴을 찾아 바꿈)
            replacement_str = rf"{values['exam_no']} \2"
            new_text = gs_report_pat.sub(replacement_str, para.text)
            
            # 문단을 지우고 새로 쓰기
            para.clear()
            run = para.add_run(new_text)
            _apply_font_style_only(run)
            
            # 기존 정렬 상태 복원
            para.alignment = original_alignment
            continue # 이 문단은 처리가 끝났으므로 다음 문단으로 넘어감

        # --- '제품명'/'버전' 치환 (기존 로직 유지) ---
        for key in REPLACEMENT_KEYS:
            value_pat = re.compile(rf"({key}\s*[:：]\s*)(.+)")
            if value_pat.search(para.text):
                original_alignment = para.alignment
                
                new_text = value_pat.sub(rf"\1{values[key]}", para.text)

                para.clear()
                run = para.add_run(new_text)
                _apply_font_style_only(run)
                para.alignment = original_alignment
                break # 키를 찾았으면 내부 루프 탈출

def _iter_paragraphs(doc):
    """문서의 본문과 표 안의 모든 문단(paragraph)을 순회하는 생성기"""
    for para in doc.paragraphs:
        yield para
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for para in cell.paragraphs:
                    yield para

In [ ]:
# =========================
# CSV 불러오기
# =========================

#--------------------경로변경---------------------------------
file_path = r"G:\내 드라이브\Auto\(25.08.21)ECM\data_B_to_P.csv"

encodings = ["utf-8-sig", "utf-8", "cp949", "euc-kr"]
for enc in encodings:
    try:
        df = pd.read_csv(file_path, encoding=enc)
        print(f"✅ CSV 불러오기 성공: {enc}")
        break
    except Exception as e:
        print(f"❌ 실패: {enc} -> {e}")

# 접수일자, 시작일, 종료일에서 " 0:00" 제거
for col in ["발급일자", "접수일자", "시작일", "종료일"]:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(" 0:00", "", regex=False)   # " 0:00" 제거
        .str.replace("-", ".", regex=False)      # "-" → "."
        .str.rstrip(".") + "."                   # 마지막에 "." 추가 (중복 방지 후 추가)
    )

# 버전 정보 추출
df["버전"] = df["시료명"].str.extract(
    r'((?:[Vv]er\.?\s*|[Vv]|버전\s*)\s*\d+(?:\.\d+)*[A-Za-z0-9]*)'
)


# 시료명에서 끝의 버전 꼬리 제거하여 '시료명 1' 생성
VERSION_TAIL_RE = r"""
    \s*
    (?:[\(\[]\s*)?
    (?:ver\.?\s*|v|V|버전)\s*
    \d+(?:\.\d+)*
    [A-Za-z0-9]*
    (?:\s*[\)\]])?
    \s*
    $
"""
df["시료명 1"] = (
    df["시료명"].astype(str)
      .str.replace(VERSION_TAIL_RE, "", regex=True, flags=re.VERBOSE)
      .str.replace(r"\s+", " ", regex=True)
      .str.strip()
)

# =========================
# main 실행

#--------------------경로변경---------------------------------
word_path = r"G:\내 드라이브\Auto\(25.08.21)ECM\기록서\시험 기록서_GS-X-XX-0XXX(영남센터).docx"

for i in range(0, len(df)):
    new_path = copy_word_with_exam_number(df, i, word_path)  # 복사본 생성
    update_word_header(df, i, new_path)                      # 머리글(맑은 고딕 9pt)
    update_table_receipt_date(df, i, new_path)               # 표 '접수일자'(굴림 10pt)
    update_product_info_final(df, i, new_path)
    # 루프 내부에서 (예: 기존 flow)


# =========================
# 데이터 미리보기 (상위 10개만)
# =========================
print(tabulate(df.head(10), headers="keys", tablefmt="pretty"))


✅ CSV 불러오기 성공: utf-8-sig
📄 복사본 생성 완료: C:\Users\dlwls\tta_new\(25.08)ECM\기록서\시험 기록서_GS-C-24-0041(영남센터).docx
✏️ 머리글 수정 완료 (맑은 고딕, 9pt 적용): C:\Users\dlwls\tta_new\(25.08)ECM\기록서\시험 기록서_GS-C-24-0041(영남센터).docx
📝 표 항목 치환 완료(굴림 10pt, 중앙정렬): C:\Users\dlwls\tta_new\(25.08)ECM\기록서\시험 기록서_GS-C-24-0041(영남센터).docx
✅ (최종 수정) 제품정보/버전/타이틀 치환 및 서식 적용 완료: C:\Users\dlwls\tta_new\(25.08)ECM\기록서\시험 기록서_GS-C-24-0041(영남센터).docx
📄 복사본 생성 완료: C:\Users\dlwls\tta_new\(25.08)ECM\기록서\시험 기록서_GS-C-24-0046(영남센터).docx
✏️ 머리글 수정 완료 (맑은 고딕, 9pt 적용): C:\Users\dlwls\tta_new\(25.08)ECM\기록서\시험 기록서_GS-C-24-0046(영남센터).docx
📝 표 항목 치환 완료(굴림 10pt, 중앙정렬): C:\Users\dlwls\tta_new\(25.08)ECM\기록서\시험 기록서_GS-C-24-0046(영남센터).docx
✅ (최종 수정) 제품정보/버전/타이틀 치환 및 서식 적용 완료: C:\Users\dlwls\tta_new\(25.08)ECM\기록서\시험 기록서_GS-C-24-0046(영남센터).docx
📄 복사본 생성 완료: C:\Users\dlwls\tta_new\(25.08)ECM\기록서\시험 기록서_GS-C-24-0054(영남센터).docx
✏️ 머리글 수정 완료 (맑은 고딕, 9pt 적용): C:\Users\dlwls\tta_new\(25.08)ECM\기록서\시험 기록서_GS-C-24-0054(영남센터).docx
📝 표 항목 치환 완료(굴림 10pt, 중앙